In [1]:
import reflex as rx
from sqlmodel import create_engine, Session, select
from weightsapp import LoggedExercise

In [2]:
sqlite_fn = "/home/mattc/code/logit/data.db"
sqlite_url = f"sqlite:///{sqlite_fn}"

engine = create_engine(sqlite_url, echo=True)


In [3]:
with Session(engine) as session:
    statement = select(LoggedExercise)
    results = session.exec(statement)
    for r in results:
        print(r)

2023-09-19 14:56:09,285 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-19 14:56:09,287 INFO sqlalchemy.engine.Engine SELECT loggedexercise.id, loggedexercise.created_datetime, loggedexercise.ename, loggedexercise.enum, loggedexercise.reps, loggedexercise.weight 
FROM loggedexercise
2023-09-19 14:56:09,287 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()
ename='Scan' enum=2 weight=70.0 reps=5 created_datetime='19-09-23' id=3
ename='Scan' enum=1 weight=70.0 reps=5 created_datetime='19-09-23' id=4
2023-09-19 14:56:09,288 INFO sqlalchemy.engine.Engine ROLLBACK


In [43]:
import pandas as pd
def _log_as_dataframe() -> pd.DataFrame:
    
    with Session(engine) as session:
        ex_list = (
            session.query(LoggedExercise).all()
        )
    ex_list = [ex._li() for ex in ex_list]    
    
    data_columns = ["date", "ename", "enum", "reps", "kg"]
    data_types = {'ename':str, 'enum': int, 'reps': int, 'kg': float}
    return pd.DataFrame(ex_list,
                            columns=data_columns).astype(data_types)

In [44]:
df = _log_as_dataframe()

2023-09-19 15:22:49,541 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-19 15:22:49,542 INFO sqlalchemy.engine.Engine SELECT loggedexercise.id AS loggedexercise_id, loggedexercise.created_datetime AS loggedexercise_created_datetime, loggedexercise.ename AS loggedexercise_ename, loggedexercise.enum AS loggedexercise_enum, loggedexercise.reps AS loggedexercise_reps, loggedexercise.weight AS loggedexercise_weight 
FROM loggedexercise
2023-09-19 15:22:49,543 INFO sqlalchemy.engine.Engine [cached since 1600s ago] ()
2023-09-19 15:22:49,544 INFO sqlalchemy.engine.Engine ROLLBACK


In [46]:
other_data = pd.DataFrame([
    ["20-09-23", "Run", "1", "10", "1000"],
    ["21-09-23", "Other", "2", "20", "500"],
    ["21-09-23", "Other", "2", "30", "500"],

                           ],
                             columns=df.columns)

df_synth = pd.concat([df, other_data])

df_synth['date'] = df_synth.date.apply(lambda x: datetime.datetime.strptime(x, "%d-%m-%y"))

In [22]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(
    data=[go.Line(x=df_synth['date'], y=df_synth['reps'])],
        layout=go.Layout(
        title=go.layout.Title(text="A Figure Specified By A Graph Object")
    )
)
fig.show()

/home/mattc/venvs/py39/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/mattc/venvs/py39/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [48]:
fig = go.Figure()
df_synth[df_synth.ename=='Other'].groupby('date').sum()


,ename,enum,reps,kg
date,,,,
2023-09-21,OtherOther,22,2030,500500


In [32]:
df_synth.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7 non-null      datetime64[ns]
 1   ename   7 non-null      object        
 2   enum    7 non-null      object        
 3   reps    7 non-null      object        
 4   kg      7 non-null      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 336.0+ bytes
